In [ ]:
!pip install ipywidgets
!pip install sqlalchemy
!pip install sqlite-utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2
!pip install streamlit PyPDF2
!pip install pytesseract
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-eng tesseract-ocr-hin tesseract-ocr-mar
!pip install python-dotenv
!pip install groq



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
The following NEW packages will be installed:
  tesseract-ocr-hin tesseract-ocr-mar
0 upgraded, 2 newly installed, 0 to remove and 1 not upgraded.
Need to get 1,775 kB of archives.
After this operation, 3,271 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-

In [ ]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install python-dotenv
!pip install calmap



In [ ]:
import sqlite3
from google.colab import drive

drive.mount('/content/drive')

conn = sqlite3.connect('/content/drive/MyDrive/studytrack.db', check_same_thread=False)
cur = conn.cursor()



Mounted at /content/drive


In [ ]:
%%writefile .env
GROQ_API_KEY="YOUR_GROQ_API_KEY"


Writing .env


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.getenv("YOUR_GROQ_API_KEY")

In [ ]:
%%writefile app.py
import streamlit as st
import sqlite3
import pandas as pd
import os

# -----------------------------
# PAGE CONFIG
# -----------------------------
st.set_page_config(
    page_title="StudyTrack AI",
    page_icon="📚",
    layout="wide"
)

# -------------- CUSTOM CSS --------------
page_bg = """
<style>
/* ----------------------------
   BACKGROUND SECTION
----------------------------- */
[data-testid="stAppViewContainer"] {
    background: url("https://img.freepik.com/free-photo/back-school-witch-school-supplies_23-2148151075.jpg?semt=ais_se_enriched&w=740&q=80");
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
}

/* Center container */
.hero-container {
    text-align: center;
    padding-top: 150px;
}

/* ----------------------------
   GLASSMORPHISM LOGIN BOX
----------------------------- */
.glass-box {
    background: rgba(255, 255, 255, 0.18);
    padding: 30px;
    width: 420px;
    margin: 60px auto;
    border-radius: 20px;
    backdrop-filter: blur(12px);
    animation: fadeIn 2.5s ease-in-out;
    text-align: center;
}

/* Login button */
.login-btn {
    background: #000000;
    color: black;
    padding: 12px 30px;
    border-radius: 30px;
    font-weight: bold;
    text-decoration: none;
    font-size: 18px;
    display: inline-block;
    transition: 0.3s;
    cursor: pointer;
    border: none;
}
.center-box {
    width: 100%;
    display: flex;
    justify-content: center;
    align-items: center;
    margin-top: 20px;
}
.login-btn:hover {
    background: #333333;
    transform: scale(1.05);
}
</style>
"""

st.markdown(page_bg, unsafe_allow_html=True)
st.markdown("""
<style>
div.stButton > button {
    display: block;
    margin-left: auto;
    margin-right: auto;
    background-color: black !important;
    color: white !important;
    padding: 15px 40px;
    border-radius: 30px;
    font-size: 20px;
    font-weight: bold;
}

div.stButton {
    text-align: center !important;
}
</style>
""", unsafe_allow_html=True)


# -----------------------------
# DB CONNECTION
# -----------------------------
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/studytrack.db')
cur = conn.cursor()

# STUDENTS TABLE
cur.execute("""
CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT UNIQUE,
    password TEXT,
    role TEXT
);
""")

# STUDY LOG TABLE (Updated with day + method)
cur.execute("""
CREATE TABLE IF NOT EXISTS study_log (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_email TEXT,
    subject TEXT,
    hours INTEGER,
    date TEXT,
    day TEXT,
    method TEXT,
    distraction_time INTEGER,
    quiz_score INTEGER,
    FOREIGN KEY (student_email) REFERENCES students (email)
);
""")
conn.commit()
print("Tables created successfully!")

# Add columns if missing
cur.execute("PRAGMA table_info(study_log)")
columns = [col[1] for col in cur.fetchall()]

if "student_id" not in columns:
    cur.execute("ALTER TABLE study_log ADD COLUMN student_id INTEGER")
    conn.commit()

if "day" not in columns:
    cur.execute("ALTER TABLE study_log ADD COLUMN day TEXT")

if "method" not in columns:
    cur.execute("ALTER TABLE study_log ADD COLUMN method TEXT")

if "distraction_time" not in columns:
    cur.execute("ALTER TABLE study_log ADD COLUMN distraction_time REAL")

if "quiz_score" not in columns:
    cur.execute("ALTER TABLE study_log ADD COLUMN quiz_score REAL")

if "student_email" not in columns:
    cur.execute("ALTER TABLE study_log ADD COLUMN student_email TEXT")

conn.commit()

# -----------------------------
# SESSION STATE
# -----------------------------
if "page" not in st.session_state:
    st.session_state.page = "home"

if "logged_email" not in st.session_state:
    st.session_state.logged_email = None

if "current_form" not in st.session_state:
    st.session_state.current_form = "login"

# -----------------------------
# FUNCTIONS
# -----------------------------
def register_student(name, email, password):
    try:
        cur.execute("INSERT INTO students (name, email, role, password) VALUES (?, ?, ?, ?)",
                    (name, email, role, password))
        conn.commit()
        return True
    except:
        return False

def check_student_login(email, password):
    cur.execute("SELECT * FROM students WHERE email=? AND password=?", (email, password))
    return cur.fetchone()

def save_log(email, student_id, subject, hours, date, day, method, distraction_time, quiz_score):
    # ---- DATABASE INSERT ----
    cur.execute("""
        INSERT INTO study_log
        (student_email, student_id, subject, hours, date, day, method, distraction_time, quiz_score)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (email, student_id, subject, hours, date, day, method, distraction_time, quiz_score))
    conn.commit()

    # ---- CSV SAVE ----
    csv_path = "/content/drive/MyDrive/study_log.csv"

    new_row = {
        "student_email": email,
        "student_id": student_id,
        "subject": subject,
        "study_hour": hours,
        "date": date,
        "day_name": day,
        "method_used": method,
        "distraction_time": distraction_time,
        "quiz_score": quiz_score
    }

    try:
        df = pd.read_csv(csv_path)
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    except:
        df = pd.DataFrame([new_row])

    df.to_csv(csv_path, index=False)


def get_student_logs(email):
    return pd.read_sql_query(
        "SELECT * FROM study_log WHERE student_email = ?",
        conn,
        params=(email,)
    )

# -----------------------------
# ADMIN FUNCTIONS
# -----------------------------
ADMIN_USER = "sneha"
ADMIN_PASS = "1234"

def admin_login(user, password):
    return user == ADMIN_USER and password == ADMIN_PASS

def admin_get_students():
    return pd.read_sql_query("SELECT * FROM students", conn)

def admin_delete_student(email):
    cur.execute("DELETE FROM students WHERE email=?", (email,))
    conn.commit()

def admin_update_student(email, new_name, new_role):
    cur.execute("UPDATE students SET name=?, role=? WHERE email=?", (new_name, new_role, email))
    conn.commit()

def export_students_to_csv():
    df_db = admin_get_students()   # DB se data laa raha hai
    df_db.to_csv("/content/drive/MyDrive/students.csv", index=False)

def get_student_logs(email):
    query = "SELECT * FROM study_log WHERE student_email = ?"
    df = pd.read_sql(query, conn, params=(email,))
    return df

# =======================
# HOME PAGE
# =======================
if st.session_state.page == "home":

    # Add custom CSS for perfect centering
    st.markdown("""
    <style>
    /* Title Animation + Gradient */
    .hero-title {
        font-size: 75px;
        font-weight: 900;
        background: linear-gradient(90deg, #000000, #3a3a3a, #000000);
        -webkit-background-clip: text;
        background-clip: text;
        color: transparent !important;
        animation: fadeIn 2s ease-in-out;
        font-family: 'Segoe UI', sans-serif;
        text-align: center;
        margin-top: 50px;
        margin-bottom: 10px;
    }

    /* Subtitle */
    .hero-subtitle {
        font-size: 28px;
        font-weight: 600;
        color: #222;
        text-align: center;
        margin-bottom: 40px;
        animation: fadeIn 2s ease-in-out;
        font-family: 'Segoe UI', sans-serif;
    }

    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(20px); }
        to { opacity: 1; transform: translateY(0); }
    }
    .hero-title {
        font-size: 75px;
        font-weight: 900;
        background: linear-gradient(90deg, #000000, #3a3a3a, #000000);
        -webkit-background-clip: text;
        background-clip: text;
        color: transparent !important;
        text-align: center;
        margin-top: 60px;
    }

    .hero-subtitle {
        font-size: 28px;
        font-weight: 600;
        color: #111;
        text-align: center;
        margin-bottom: 40px;
    }

    /* Button Styling */
    div.stButton > button {
        background-color: black !important;
        color: white !important;
        padding: 15px 40px !important;
        font-size: 22px !important;
        border-radius: 30px !important;
        border: none !important;
        font-weight: bold !important;
        transition: 0.3s ease;
    }

    div.stButton > button:hover {
        background-color: #333 !important;
        transform: scale(1.05);
    }
    /* Section Header */
    .section-header {
        text-align: center;
        font-size: 42px;
        font-weight: 800;
        color: #2c3e50;
        margin: 40px 0 60px 0;
        font-family: 'Segoe UI', sans-serif;
        position: relative;
        padding-bottom: 15px;
    }

    .section-header::after {
        content: '';
        position: absolute;
        bottom: 0;
        left: 50%;
        transform: translateX(-50%);
        width: 100px;
        height: 4px;
        background: linear-gradient(90deg, #000000, #3a3a3a, #000000);
        border-radius: 2px;
    }

    /* Feature Cards Container */
    .features-container {
        display: flex;
        justify-content: center;
        gap: 50px;
        margin: 40px 0 80px 0;
        flex-wrap: wrap;
        padding: 0 20px;
    }

    /* TRANSPARENT CARDS - Blend with background */
    .feature-card {
        background: rgba(255, 255, 255, 0.1) !important;  /* More transparent */
        backdrop-filter: blur(20px) !important;
        -webkit-backdrop-filter: blur(20px) !important;
        border-radius: 25px;
        padding: 40px 30px;
        width: 350px;
        text-align: center;
        box-shadow:
            0 8px 32px rgba(0, 0, 0, 0.08),
            inset 0 1px 0 rgba(255, 255, 255, 0.2);
        transition: all 0.4s cubic-bezier(0.175, 0.885, 0.32, 1.275);
        border: 1px solid rgba(255, 255, 255, 0.2);
        position: relative;
        overflow: hidden;
    }

    /* Optional: Very subtle gradient overlay */
    .feature-card::before {
        content: '';
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        background: linear-gradient(
            135deg,
            rgba(255, 255, 255, 0.05) 0%,
            rgba(255, 255, 255, 0.01) 100%
        );
        z-index: -1;
        border-radius: 25px;
    }

    .feature-card:hover {
        transform: translateY(-10px);
        box-shadow:
            0 20px 40px rgba(0, 0, 0, 0.12),
            inset 0 1px 0 rgba(255, 255, 255, 0.3);
        background: rgba(255, 255, 255, 0.15) !important;
        border: 1px solid rgba(255, 255, 255, 0.3);
    }

    /* Card Icons - Interactive */
    .card-icon {
        font-size: 70px;
        margin-bottom: 25px;
        display: inline-block;
        transition: all 0.4s ease;
        filter: drop-shadow(0 5px 10px rgba(0, 0, 0, 0.15));
        color: #2c3e50;
    }

    .feature-card:hover .card-icon {
        transform: scale(1.15) rotate(5deg);
        filter: drop-shadow(0 8px 15px rgba(0, 0, 0, 0.25));
    }

    /* Card Title */
    .card-title {
        font-size: 28px;
        font-weight: 800;
        color: #2c3e50;
        margin-bottom: 20px;
        font-family: 'Segoe UI', sans-serif;
        letter-spacing: -0.5px;
        text-shadow: 0 1px 2px rgba(0, 0, 0, 0.1);
    }

    /* Card Description */
    .card-desc {
        font-size: 18px;
        color: #4a5568;
        line-height: 1.6;
        font-family: 'Segoe UI', sans-serif;
        padding: 0 10px;
        text-shadow: 0 1px 1px rgba(255, 255, 255, 0.5);
    }

    /* Subtle accent line */
    .card-hover-line {
        height: 2px;
        width: 0;
        background: linear-gradient(90deg, transparent, rgba(44, 62, 80, 0.3), transparent);
        margin: 25px auto 0 auto;
        transition: width 0.4s ease;
        border-radius: 2px;
    }

    .feature-card:hover .card-hover-line {
        width: 60%;
    }

    /* Responsive adjustments */
    @media (max-width: 1200px) {
        .feature-card {
            width: 320px;
        }
    }

    @media (max-width: 992px) {
        .features-container {
            gap: 40px;
        }
        .feature-card {
            width: 300px;
        }
    }

    @media (max-width: 768px) {
        .feature-card {
            width: 100%;
            max-width: 400px;
            background: rgba(255, 255, 255, 0.12) !important;
        }
        .hero-title {
            font-size: 55px;
        }
        .hero-subtitle {
            font-size: 24px;
        }
    }

    /* Make the background more visible for testing */
    .stApp {
        background: linear-gradient(135deg, #f5f7fa 0%, #e4e9f2 100%);
        min-height: 100vh;
    }

    </style>
    """, unsafe_allow_html=True)

    # Titles
    st.markdown('<div class="hero-title">Welcome to StudyTrack AI</div>', unsafe_allow_html=True)
    st.markdown('<div class="hero-subtitle">Smart Study Habit Recommendation System</div>', unsafe_allow_html=True)

    # ----------------------------------
    # PERFECT CENTER BUTTON USING COLUMNS
    # ----------------------------------
    col1, col2, col3 = st.columns([3, 1, 2.5])

    with col2:  # button exactly middle me
        if st.button("Login"):
            st.session_state.page = "login_register_admin"
            st.rerun()

     # Add spacing after cards
    st.markdown("<br><br><br><br><br>", unsafe_allow_html=True)
    # ==========================================
    # 3 FEATURE CARDS BELOW THE LOGIN BUTTON
    # ==========================================
    st.markdown("<br><br>", unsafe_allow_html=True)
    st.markdown('<div class="section-header">Key Features</div>', unsafe_allow_html=True)

    # Create three columns for the cards with proper spacing
    col1, col2, col3 = st.columns([1, 1, 1], gap="large")

    with col1:
        st.markdown("""
        <div class="feature-card">
            <div class="card-icon">📊</div>
            <div class="card-title">AI-Powered Analytics</div>
            <div class="card-desc">
                Get personalized insights into your study patterns with advanced
                machine learning algorithms that identify your strengths and areas
                for improvement in real-time.
            </div>
            <div class="card-hover-line"></div>
        </div>
        """, unsafe_allow_html=True)

    with col2:
        st.markdown("""
        <div class="feature-card">
            <div class="card-icon">🔍</div>
            <div class="card-title">Smart Recommendations</div>
            <div class="card-desc">
                Receive customized study plans and habit suggestions tailored to your
                learning style, schedule, and academic goals. Adapts as you progress.
            </div>
            <div class="card-hover-line"></div>
        </div>
        """, unsafe_allow_html=True)

    with col3:
        st.markdown("""
        <div class="feature-card">
            <div class="card-icon">📈</div>
            <div class="card-title">Progress Tracking</div>
            <div class="card-desc">
                Visualize your learning journey with detailed progress reports,
                achievement badges, and predictive performance analytics. Stay motivated!
            </div>
            <div class="card-hover-line"></div>
        </div>
        """, unsafe_allow_html=True)

    # Add final spacing
    st.markdown("<br><br><br>", unsafe_allow_html=True)

# =======================
# LOGIN / REGISTER / ADMIN PAGE
# =======================

elif st.session_state.page == "login_register_admin":

    # Initialize active tab in session state
    if "active_tab" not in st.session_state:
        st.session_state.active_tab = "login"

    # ------------------ LOGIN PAGE CSS ------------------
    login_page_css = """
    <style>
    /* FIXED TOP RIGHT BUTTON BAR */
     .fixed-nav-box {
        position: fixed;
        top: 20px;
        right: 30px;
        z-index: 99999;
        display: flex;
        gap: 12px;
        padding: 10px 20px;
        background: rgba(255,255,255,0.20);
        backdrop-filter: blur(10px);
        border-radius: 40px;
        box-shadow: 0 4px 20px rgba(0,0,0,0.2);
    }
    .fixed-nav-box button {
        font-size: 15px !important;
        font-weight: 600 !important;
        padding: 6px 20px !important;
        border-radius: 10px !important;
    }

    .nav-btn-container {
        display: flex;
        gap: 10px;
    }

    /* Form Container */
    .form-wrapper {
        width: 500px;
        margin: 120px auto 50px;
        padding: 40px;
        background: rgba(255, 255, 255, 0.18);
        border-radius: 25px;
        backdrop-filter: blur(10px);
        border: 1px solid rgba(255, 255, 255, 0.2);
        box-shadow: 0 20px 40px rgba(0,0,0,0.15);
    }

    .form-title {
        color: #000000;
        text-align: center;
        font-size: 32px;
        margin-bottom: 30px;
        font-weight: 700;
    }

    /* Inputs */
    .stTextInput>div>div>input,
    .stSelectbox>div>div>select {
        background: rgba(255, 255, 255, 0.9) !important;
        border: 2px solid rgba(0, 0, 0, 0.15) !important;
        color: #000000 !important;
        border-radius: 12px !important;
        padding: 14px !important;
        font-size: 18px !important;
        font-weight: 500 !important;
    }

    /* Labels */
    .stTextInput label,
    .stSelectbox label {
        color: #000000 !important;
        font-weight: 600 !important;
        font-size: 18px !important;
        margin-bottom: 8px !important;
    }

    /* Form buttons */
    .stButton > button {
        background: #000000 !important;
        color: white !important;
        border: none !important;
        border-radius: 12px !important;
        padding: 16px !important;
        font-size: 20px !important;
        font-weight: 600 !important;
        transition: all 0.3s !important;
        width: 100% !important;
        margin-top: 20px !important;
    }

    .stButton > button:hover {
        background: #333333 !important;
        transform: translateY(-3px) !important;
    }

    /* Back to Home Button */
    .back-home-container {
        text-align: center;
        margin-top: 40px;
        margin-bottom: 20px;
    }
    </style>
    """

    st.markdown(login_page_css, unsafe_allow_html=True)

    # =======================
    # TOP RIGHT NAVIGATION BUTTONS
    # =======================
    # Create container for top right buttons
    st.markdown("""
    <div class="top-right-nav">
        <div class="nav-btn-container">
    """, unsafe_allow_html=True)

    # Use Streamlit columns for proper layout
    col1, col2, col3, col4 = st.columns([9, 0.7, 0.8, 0.8])

    with col2:
        if st.button("Login", key="top_login_btn",
                    help="Student Login",
                    type="primary" if st.session_state.active_tab == "login" else "secondary"):
            st.session_state.active_tab = "login"
            st.rerun()

    with col3:
        if st.button("Register", key="top_register_btn",
                    help="Create Account",
                    type="primary" if st.session_state.active_tab == "register" else "secondary"):
            st.session_state.active_tab = "register"
            st.rerun()

    with col4:
        if st.button("Admin", key="top_admin_btn",
                    help="Admin Login",
                    type="primary" if st.session_state.active_tab == "admin" else "secondary"):
            st.session_state.active_tab = "admin"
            st.rerun()

    st.markdown("""
        </div>
    </div>
    """, unsafe_allow_html=True)

    # =======================
    # MAIN FORM CONTENT
    # =======================
    # STUDENT LOGIN PAGE
    if st.session_state.active_tab == "login":
        st.markdown("<h1 style='font-size:80px; color:black; text-align:center;'>👤</h1>", unsafe_allow_html=True)
        st.markdown("<h1 style='color:black;text-align:center;'>Welcome to Student Login</h1>", unsafe_allow_html=True)
        st.markdown("<h5 style='color:black;text-align:center;'>Smart Study Habit Recommendation System</h5>", unsafe_allow_html=True)

        email = st.text_input("📧 Email")
        password = st.text_input("🔒 Password", type="password")
        role = st.selectbox("🎯 Role", ["Student", "Teacher", "Admin"])

        if st.button("Login", key="login_btn", use_container_width=True, type="primary"):
            student = check_student_login(email, password)
            if student:
                st.session_state.logged_email = email
                st.session_state.page = "student_dashboard"
                st.rerun()
            else:
                st.error("❌ Invalid Login Details")

        st.markdown("""
        <div style="text-align: center; margin-top: 20px; color: #000000; font-size: 17px;">
            Don't have an account?
            <span onclick="document.getElementById('top_register_btn').click()"
                  style="color: #000000; font-weight: 700; cursor: pointer; text-decoration: underline;">
                Register here
            </span>
        </div>
        """, unsafe_allow_html=True)

    # REGISTER PAGE
    elif st.session_state.active_tab == "register":
        st.markdown("<h1 style='font-size:80px; color:black; text-align:center;'>👤</h1>", unsafe_allow_html=True)
        st.markdown("<h1 style='color:black;text-align:center;'>Create Student Account</h1>", unsafe_allow_html=True)
        st.markdown("<h5 style='color:black;text-align:center;'>Smart Study Habit Recommendation System</h5>", unsafe_allow_html=True)

        name = st.text_input("👤 Full Name")
        email = st.text_input("📧 Email")
        role = st.selectbox("🎯 Role", ["Student", "Teacher", "Admin"])
        gender = st.selectbox("⚧ Gender", ["Male", "Female"])
        password = st.text_input("🔒 Password", type="password")
        confirm_password = st.text_input("✓ Confirm Password", type="password")

        if st.button("Register", key="register_btn", use_container_width=True, type="primary"):
            if password != confirm_password:
                st.error("❌ Passwords do not match")
            elif not all([name, email, password]):
                st.error("Please fill all required fields!")
            else:
                if register_student(name, email, password):
                    st.success("🎉 Registration Successful! Please login.")
                    st.session_state.active_tab = "login"
                    st.rerun()
                else:
                    st.error("Email already exists!")

        st.markdown("""
        <div style="text-align: center; margin-top: 20px; color: #000000; font-size: 17px;">
            Already have an account?
            <span onclick="document.getElementById('top_login_btn').click()"
                  style="color: #000000; font-weight: 700; cursor: pointer; text-decoration: underline;">
                Login here
            </span>
        </div>
        """, unsafe_allow_html=True)

    # ADMIN LOGIN PAGE
    elif st.session_state.active_tab == "admin":
        st.markdown("<h1 style='font-size:80px; color:black; text-align:center;'>👤</h1>", unsafe_allow_html=True)
        st.markdown("<h1 style='color:black;text-align:center;'>Welcome to Admin Login</h1>", unsafe_allow_html=True)
        st.markdown("<h5 style='color:black;text-align:center;'>Smart Study Habit Recommendation System</h5>", unsafe_allow_html=True)

        username = st.text_input("👤 Admin Username")
        password = st.text_input("🔒 Admin Password", type="password")

        if st.button("Admin Login", key="admin_btn", use_container_width=True, type="primary"):
            if admin_login(username, password):
                st.session_state.page = "admin_dashboard"
                st.rerun()
            else:
                st.error("❌ Invalid Admin Credentials")

        st.markdown("""
        <div style="text-align: center; margin-top: 20px; color: #000000; font-size: 17px;">
            Student login?
            <span onclick="document.getElementById('top_login_btn').click()"
                  style="color: #000000; font-weight: 700; cursor: pointer; text-decoration: underline;">
                Click here
            </span>
        </div>
        """, unsafe_allow_html=True)

    st.markdown('</div>', unsafe_allow_html=True)

    # BOTTOM BACK TO HOME BUTTON - CENTERED AT BOTTOM
    st.markdown("""
    <div class="bottom-back-container">
    """, unsafe_allow_html=True)

    # Add some space
    for _ in range(5):
        st.write("")

    # Create centered button at bottom
    col1, col2, col3 = st.columns([2, 1, 2])
    with col2:
        if st.button("← Back to Home", key="bottom_back_home",
                    use_container_width=True, type="secondary"):
            st.session_state.page = "home"
            st.rerun()

    st.markdown("</div>", unsafe_allow_html=True)


# ======================================================
# STUDENT DASHBOARD
# ======================================================
elif st.session_state["page"] == "student_dashboard":

    st.markdown("<h1 style='color:black;'>🎓 Student Dashboard</h1>", unsafe_allow_html=True)
    st.success(f"Logged in as: {st.session_state['logged_email']}")

    tab1, tab2, tab3, tab4 = st.tabs(["➕ Add Study Log", "📄 My Study Logs", "📝 Test Model", "📊 Performance"])

    # ADD LOG
    with tab1:
        st.markdown("<div class='glass-card'>", unsafe_allow_html=True)

        student_id = st.text_input("Student ID")
        subject = st.selectbox("Subject", ["Maths", "English", "Science", "Computer", "History"])
        hours = st.number_input("Hours Studied", min_value=0)
        method = st.selectbox("Study Method",
            ["Videos", "Notes Review", "Practice Test", "Group Study", "Flashcards", "pomodoro"])
        distraction_time = st.number_input("Distraction Time (in minutes)", min_value=0)
        quiz_score = st.number_input("Quiz Score", min_value=0)

        day = st.selectbox("Day",
            ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

        date = st.date_input("Date")

        if st.button("Save Log"):
            save_log(
                st.session_state["logged_email"],
                student_id,
                subject,
                hours,
                str(date),
                day,
                method,
                distraction_time,
                quiz_score
            )
            st.success("📌 Study Log Saved!")

    st.markdown("</div>", unsafe_allow_html=True)

# VIEW LOGS
    with tab2:
        email = st.session_state.logged_email

        st.markdown("### View Study Logs")

        # ---------- DATABASE DATA ----------
        try:
            df_db = pd.read_sql_query(
                "SELECT date, subject, hours, day, method, distraction_time, quiz_score "
                "FROM study_log WHERE student_email = ?",
                conn,
                params=(email,)
            )

            if df_db.empty:
                st.info("No logs found in database")
            else:
                st.dataframe(df_db)

        except Exception as e:
            st.error(f"DB Error: {e}")

        st.markdown("</div>", unsafe_allow_html=True)

    # Test model

    with tab3:
        st.markdown("### 📝 AI Test Generator")
        st.write("Upload your notes and let the AI generate a customizable test.")

        uploaded_file = st.file_uploader("Upload Notes (Text / PDF / Image)",
                                        type=["txt", "pdf", "png", "jpg", "jpeg"])

        difficulty = st.selectbox("Select Difficulty", ["Easy", "Medium", "Hard"])

        generate_btn = st.button("Generate Test")


        # -------------------- AI GENERATOR --------------------
        if generate_btn and uploaded_file:

            import PyPDF2
            import pytesseract
            from PIL import Image
            from dotenv import load_dotenv
            import os
            from groq import Groq

            load_dotenv()

            # Load Groq API key from .env
            client = Groq(api_key=os.getenv("GROQ_API_KEY"))


            # ---- TEXT EXTRACTION ----
            def extract_text(file):
                if file.type == "text/plain":
                    return file.read().decode("utf-8")

                elif file.type == "application/pdf":
                    reader = PyPDF2.PdfReader(file)
                    text = ""
                    for page in reader.pages:
                        if page.extract_text():
                            text += page.extract_text()
                    return text

                else:  # image OCR
                    image = Image.open(file)
                    return pytesseract.image_to_string(image)


            notes_text = extract_text(uploaded_file)


            # ---------------- Prompt ----------------
            prompt = f"""
            You are an AI Test Generator.
            Create a {difficulty} level MCQ test from the notes below.

            STRICT FORMAT:
            Q1. Question text
            A) option
            B) option
            C) option
            D) option
            Correct: A

            Notes:
            {notes_text}
            """


            # ---------------- Groq Correct Model ----------------
            response = client.chat.completions.create(
                model="llama-3.1-8b-instant",   # ✅ NEW SUPPORTED MODEL
                messages=[{"role": "user", "content": prompt}]
            )

            generated_test = response.choices[0].message.content

            st.session_state.generated_test = generated_test


        # -------------------- DISPLAY TEST --------------------
        if "generated_test" in st.session_state:

            st.markdown("## 🖊 Attempt Your Test")

            test_text = st.session_state.generated_test.split("\n")

            questions = []
            options = []
            answers = []

            q = ""
            opt = []
            corr = ""

            for line in test_text:
                line = line.strip()

                if line.startswith("Q"):
                    if q:
                        questions.append(q)
                        options.append(opt)
                        answers.append(corr)
                        opt = []
                    q = line

                elif line.startswith(("A)", "B)", "C)", "D)")):
                    opt.append(line)

                elif "Correct:" in line:
                    corr = line.split(":")[1].strip()

            if q:
                questions.append(q)
                options.append(opt)
                answers.append(corr)

            user_answers = {}

            for i, q_text in enumerate(questions):
                st.write(f"### {q_text}")
                user_answers[i] = st.radio(
                    "Select an option",
                    options[i],
                    key=f"q{i}",
                    index=None
                )
                st.write("---")

            if st.button("Submit Test"):
                score = 0
                st.markdown("## 📊 Test Results")

                for i, ua in user_answers.items():
                    correct = answers[i]
                    correct_full = next((o for o in options[i] if o.startswith(correct)), "N/A")

                    if ua and ua.startswith(correct):
                        st.success(f"{questions[i]} ✔ Correct")
                        score += 1
                    else:
                        st.error(f"{questions[i]} ❌ Wrong\nCorrect Answer: {correct_full}")

                st.markdown(f"### 🏆 Total Score: **{score}/{len(questions)}**")

#  -----------------------------------------------
    # ********* PERFORMANCE TAB *********
#  -----------------------------------------------
    with tab4:
        st.markdown("""
        <style>

        /* CARD GRID */
        .card-grid {
            display: grid;
            grid-template-columns: repeat(2, 1fr);
            gap: 20px;
            margin-top: 20px;
        }

        /* SINGLE CARD */
        .ai-card {
            background: rgba(255, 255, 255, 0.15);
            backdrop-filter: blur(10px);
            border-radius: 16px;
            padding: 20px;
            box-shadow: 0 8px 25px rgba(0,0,0,0.25);
            min-height: 120px;
        }

        /* CARD TITLE */
        .ai-card h4 {
            margin-bottom: 10px;
            color: black;
        }

        /* CARD VALUE */
        .ai-card p {
            font-size: 22px;
            font-weight: 600;
            margin: 0;
        }

        </style>
        """, unsafe_allow_html=True)

        st.markdown("<h2 style='color:black;'>📊 Personalized AI Study Recommendations</h2>", unsafe_allow_html=True)

        df = get_student_logs(st.session_state["logged_email"])

        if df.empty:
            st.warning("No study logs found. Add study logs first!")

        else:
            # ---------------- TOP, WEAK, RECOMMENDED SUBJECTS ----------------
            total_hours = df.groupby("subject")["hours"].sum()

            top_subject = total_hours.idxmax()
            weak_subject = total_hours.idxmin()
            recommended_subject = df['subject'].value_counts().idxmin()

            # ---------------- SUBJECT SCORE ----------------
            subject_score = {}
            max_hours = total_hours.max()

            for sub, hours in total_hours.items():
                if max_hours == 0:
                    score = 0
                else:
                    score = int((hours / max_hours) * 100)

                subject_score[sub] = score

            # ---------------- AI ADVICE ----------------
            def get_advice(score):
                if score >= 80:
                    return "🌟 Excellent performance! Keep dominating!"
                elif score >= 50:
                    return "👍 Good work! Keep improving!"
                else:
                    return "⚠ Needs improvement — Work harder!"

            if subject_score:
                avg_score = int(sum(subject_score.values()) / len(subject_score))
                final_advice = get_advice(avg_score)
            else:
                avg_score = 0
                final_advice = "No study activity yet."

            # ---------------- DISPLAY CARDS ----------------
            st.markdown("### Your AI Recommendations")

            col1, col2 = st.columns(2)

            with col1:
                st.markdown(
                    f"<div class='ai-card'><h4>🏆 Top Performing Subject</h4>"
                    f"<p style='font-size:20px; color:#2E8B57;'>{top_subject}</p></div>",
                    unsafe_allow_html=True
                )

                st.markdown(
                    f"<div class='ai-card'><h4>📘 Recommended Subject</h4>"
                    f"<p style='font-size:20px; color:#2E8B57;'>{recommended_subject}</p></div>",
                    unsafe_allow_html=True
                )

            with col2:
                st.markdown(
                    f"<div class='ai-card'><h4>⚠ Weak Subject</h4>"
                    f"<p style='font-size:20px; color:#2E8B57;'>{weak_subject}</p></div>",
                    unsafe_allow_html=True
                )

                st.markdown(
                    f"<div class='ai-card'><h4>🧠 Personalized Advice</h4>"
                    f"<p style='font-size:20px; color:#2E8B57;'>{final_advice}</p></div>",
                    unsafe_allow_html=True
                )

            # ---------------- SUBJECT-WISE SCORE ----------------
            st.markdown("<h3 style='color:black;'>📈 Subject-wise Performance Score</h3>", unsafe_allow_html=True)

            score_df = pd.DataFrame({
                "Subject": list(subject_score.keys()),
                "Score": list(subject_score.values())
            })

            st.bar_chart(score_df, x="Subject", y="Score", height=300)

            # ---------------- STUDY HEATMAP ----------------

            import pandas as pd
            import streamlit as st
            import altair as alt

            # ------------------ TITLE ------------------
            st.markdown(
                "<h3 style='color:black; margin-top:20px;'>🔥 Study Contribution Grid</h3>",
                unsafe_allow_html=True
            )

            # ------------------ COLOR LEGEND (HTML + CSS) ------------------
            legend_html = """
            <style>
            .legend-box {
                display: flex;
                gap: 20px;
                margin: 15px 0 30px 5px;
            }

            .legend-item {
                display: flex;
                align-items: center;
                gap: 8px;
                font-size: 14px;
                color: black;
            }

            .color-sq {
                width: 18px;
                height: 18px;
                border-radius: 4px;
                display: inline-block;
            }
            </style>

            <div class="legend-box">
                <div class="legend-item">
                    <span class="color-sq" style="background:#9be9a8;"></span> Light Green - Low Study
                </div>
                <div class="legend-item">
                    <span class="color-sq" style="background:#40c463;"></span> Green - Moderate Study
                </div>
                <div class="legend-item">
                    <span class="color-sq" style="background:#216e39;"></span> Dark Green - High Study
                </div>
            </div>
            """

            st.markdown(legend_html, unsafe_allow_html=True)

            # ------------------ MAIN HEATMAP CODE ------------------
            try:
                df["date"] = pd.to_datetime(df["date"], errors="coerce")
                df = df.dropna(subset=["date"])

                full = pd.DataFrame({"date": pd.date_range(df["date"].min(), df["date"].max())})
                full = full.merge(df, on="date", how="left")
                full["hours"] = full["hours"].fillna(0)

                full["dow"] = full["date"].dt.weekday
                full["week"] = full["date"].dt.isocalendar().week
                full["month"] = full["date"].dt.month
                full["month_name"] = full["date"].dt.strftime("%b")
                full["year"] = full["date"].dt.year

                full["week_index"] = (full["date"] - full["date"].min()).dt.days // 7

                full["level"] = pd.cut(
                    full["hours"],
                    bins=[-1, 0, 1, 2, 4, 100],
                    labels=[0, 1, 2, 3, 4]
                ).astype(int)

                colors = ["#ebedf0", "#9be9a8", "#40c463", "#30a14e", "#216e39"]

                heatmap = alt.Chart(full).mark_rect(
                    width=12,
                    height=12,
                    cornerRadius=3
                ).encode(
                    x=alt.X("week_index:O", title=None, axis=None),
                    y=alt.Y(
                        "dow:O",
                        sort=[0,1,2,3,4,5,6],
                        title=None,
                        axis=alt.Axis(
                            labels=True,
                            labelExpr=(
                                "datum.value == 0 ? 'Mon' : "
                                "datum.value == 3 ? 'Thu' : "
                                "datum.value == 6 ? 'Sun' : ''"
                            )
                        )
                    ),
                    color=alt.Color("level:N", scale=alt.Scale(range=colors), legend=None),
                    tooltip=[
                        alt.Tooltip("date:T", title="Date"),
                        alt.Tooltip("hours:Q", title="Hours Studied")
                    ]
                )

                year_labels_df = full.groupby("year").first().reset_index()
                year_labels = alt.Chart(year_labels_df).mark_text(
                    align="right",
                    baseline="middle",
                    dx=-25,
                    color="black",
                    fontSize=16,
                    fontWeight="bold"
                ).encode(
                    y="dow:O",
                    text="year:N"
                )

                month_labels_df = full.groupby("month").first().reset_index()
                month_labels = alt.Chart(month_labels_df).mark_text(
                    align="center",
                    dy=-20,
                    fontSize=13,
                    color="black",
                    fontWeight="bold"
                ).encode(
                    x="week_index:O",
                    text="month_name:N"
                )

                final = (heatmap + month_labels + year_labels).properties(
                    width=1200,
                    height=190
                )

                st.altair_chart(final, use_container_width=True)

            except Exception as e:
                st.error(f"Study Heatmap Error: {e}")


            # ---------------- SCORE LABELS ---------------
            import streamlit as st
            import streamlit.components.v1 as components

            # ---------- TITLE ----------
            st.markdown("<h3 style='color:black;'>📌 Subject Ratings</h3>", unsafe_allow_html=True)

            # ---------- CSS (inject once) ----------
            css = """
            <style>
            .rating-grid {
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(260px, 1fr));
                gap: 20px;
                margin-top: 20px;
            }

            .rating-card {
                background: rgba(255, 255, 255, 0.85);
                padding: 18px;
                border-radius: 16px;
                box-shadow: 0 8px 22px rgba(0,0,0,0.12);
                min-height: 100px;
            }

            .rating-card h4 {
                margin-bottom: 10px;
                color: #000;
                font-size: 20px;
            }

            .rating-card p {
                font-size: 16px;
                margin: 0;
            }
            </style>
            """

            # ---------- BUILD CARDS HTML ----------
            cards_html = "<div>" + css  # include CSS inside the same html blob for components.html

            cards_html += "<div class='rating-grid'>"

            # subject_score should be defined already (dict like {"Math": 80, "Science": 100})
            for subject, score in subject_score.items():
                if score >= 80:
                    rating = "🌟 Excellent"
                    color = "#2E8B57"
                elif score >= 50:
                    rating = "👍 Good"
                    color = "#1E90FF"
                else:
                    rating = "⚠ Work Harder"
                    color = "#DC143C"

                # escape subject text just in case (prevent accidental html injection)
                safe_subject = str(subject).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

                cards_html += f"""
                <div class="rating-card">
                    <h4>{safe_subject}</h4>
                    <p style="color:{color}; font-weight:600;">
                        Score: {score}% — {rating}
                    </p>
                </div>
                """

            cards_html += "</div></div>"

            # ---------- RENDER USING components.html (reliable raw HTML render) ----------
            # set height dynamically: rows * 140 approx, or a fixed height
            approx_rows = max(1, (len(subject_score) + 1) // 2)  # 2 cards per row
            height_px = approx_rows * 160 + 80

            components.html(cards_html, height=height_px, scrolling=True)


    if st.button("Logout"):
        st.session_state["page"] = "home"
        st.rerun()

# ======================================================
# ADMIN DASHBOARD
# ======================================================
elif st.session_state["page"] == "admin_dashboard":

    st.markdown("<h1 style='color:black;'>🛠 Admin Panel</h1>", unsafe_allow_html=True)

    tab1, tab2, tab3, tab4 = st.tabs([
        "📋 View Students",
        "✏ Update Student",
        "🗑 Delete Student",
        "📊 Visualization"
    ])

    # VIEW
    with tab1:
        st.write("## Students Database")
        df = admin_get_students()

        # REMOVE EMPTY/NULL ROWS
        df = df.dropna(how="all")                                # full empty row delete
        df = df[df['name'].notna() & (df['name'].str.strip() != "")]   # name null ya blank ho to delete
      # Name empty rows delete

        st.dataframe(df, hide_index=True)


        st.markdown("</div>", unsafe_allow_html=True)


    # UPDATE
    with tab2:
        update_email = st.text_input("Email to Update")
        new_name = st.text_input("New Name")
        if st.button("Update"):
            admin_update_student(update_email, new_name)
            st.success("✔ Student Updated!")
        st.markdown("</div>", unsafe_allow_html=True)

    # DELETE
    with tab3:
        del_email = st.text_input("Email to Delete")
        if st.button("Delete"):
            admin_delete_student(del_email)
            st.success("🗑 Student Deleted!")
        st.markdown("</div>", unsafe_allow_html=True)

   #visulaization
    with tab4:
        import streamlit as st
        import pandas as pd
        import matplotlib.pyplot as plt

        st.markdown("""
        <style>
        .glass-card {
            background: rgba(255, 255, 255, 0.35);
            backdrop-filter: blur(20px);
            -webkit-backdrop-filter: blur(20px);
            border-radius: 24px;
            padding: 28px 22px;
            box-shadow: 0 12px 35px rgba(0,0,0,0.15);
            transition: all 0.35s ease;
            text-align: center;
        }

        .glass-card:hover {
            transform: translateY(-8px) scale(1.04);
            box-shadow: 0 20px 45px rgba(0,0,0,0.22);
        }

        .card-icon {
            font-size: 30px;
            margin-bottom: 10px;
        }

        .card-title {
            font-size: 16px;
            font-weight: 600;
            color: #374151;
        }

        .card-value {
            font-size: 42px;
            font-weight: 900;
            color: #111827;
            margin-top: 6px;
        }
        </style>
        """, unsafe_allow_html=True)

        email_input = st.text_input("Enter Student Email")

        if st.button("Fetch Data"):

            if not email_input.strip():
                st.warning("Please enter an email")
                st.stop()

            df_logs = pd.read_sql(
                """
                SELECT
                    date,
                    subject,
                    hours,
                    distraction_time,
                    quiz_score
                FROM study_log
                WHERE student_email = ?
                """,
                conn,
                params=(email_input,)
            )

            # safe column names
            df_logs.columns = df_logs.columns.str.lower()

            if df_logs.empty:
                st.error("No study activity found")
                st.stop()

            df_logs['date'] = pd.to_datetime(df_logs['date'])
            df_logs = df_logs.sort_values("date")

            total_hours = df_logs['hours'].sum()
            avg_daily = df_logs['hours'].mean()
            avg_dist = df_logs['distraction_time'].mean()
            total_logs = len(df_logs)

            # ---------- TOP METRICS ----------
            st.markdown("### Quick Stats")
            c1, c2, c3, c4 = st.columns(4)

            with c1:
                st.markdown(f"""
                <div class="glass-card">
                    <div class="card-title">Total Study Hours</div>
                    <div class="card-value">{total_hours:.1f} h</div>
                </div>
                """, unsafe_allow_html=True)

            with c2:
                st.markdown(f"""
                <div class="glass-card">
                    <div class="card-title">Avg Daily Study</div>
                    <div class="card-value">{avg_daily:.1f} h</div>
                </div>
                """, unsafe_allow_html=True)

            with c3:
                st.markdown(f"""
                <div class="glass-card">
                    <div class="card-title">Avg Distraction</div>
                    <div class="card-value">{avg_dist:.1f} min</div>
                </div>
                """, unsafe_allow_html=True)

            with c4:
                st.markdown(f"""
                <div class="glass-card">
                    <div class="card-title">Total Logs</div>
                    <div class="card-value">{total_logs}</div>
                </div>
                """, unsafe_allow_html=True)

            # ---------- MAIN CHARTS ----------
            st.markdown("### 📈 Visual Insights")
            g1, g2 = st.columns(2)

            # DAILY STUDY HOURS (LINE CHART)
            with g1:
                st.markdown("""
                <div style='background:#ffffff20; padding:15px; border-radius:15px;'>
                    <h3 style='text-align:center; color:black;'>Daily Study Hours</h3>
                </div>
                """, unsafe_allow_html=True)

                fig1, ax1 = plt.subplots(figsize=(6, 4))

                # Transparent background
                fig1.patch.set_alpha(0)
                ax1.set_facecolor("none")

                ax1.plot(df_logs['date'], df_logs['hours'], marker='o', linewidth=2)
                ax1.set_xlabel("Date")
                ax1.set_ylabel("Study Hours")
                ax1.grid(True, linestyle="--", alpha=0.3)
                plt.xticks(rotation=45)

                st.pyplot(fig1)

            # DISTRACTION TIME (BAR CHART)
            with g2:
                st.markdown("""
                <div style='background:#ffffff20; padding:15px; border-radius:15px;'>
                    <h3 style='text-align:center; color:black;'>Distraction Hours</h3>
                </div>
                """, unsafe_allow_html=True)

                fig2, ax2 = plt.subplots(figsize=(6, 4))

                fig2.patch.set_alpha(0)
                ax2.set_facecolor("none")

                ax2.hist(df_logs['hours'], bins=8, edgecolor='black', alpha=0.7)
                ax2.set_xlabel("Study Hours")
                ax2.set_ylabel("Frequency")
                ax2.grid(True, linestyle="--", alpha=0.3)

                st.pyplot(fig2)

            # ---------- EXTRA CHARTS (PIE + SCATTER) ----------
            g3, g4 = st.columns(2)

            # PIE CHART
            with g3:
                st.markdown("""
                <div style='background:#ffffff20; padding:15px; border-radius:15px;'>
                    <h3 style='text-align:center; color:black;'> Study vs Distraction Ratio</h3>
                </div>
                """, unsafe_allow_html=True)

                labels = ["Study Hours", "Distraction Time"]
                values = [total_hours, df_logs['distraction_time'].sum()]

                # ---- SMALLER PIE CIRCLE ----
                fig_pie, ax_pie = plt.subplots(figsize=(3.5, 3.5))   # smaller figure size
                fig_pie.patch.set_alpha(0)
                ax_pie.set_facecolor("none")

                ax_pie.pie(
                    values,
                    labels=labels,
                    autopct='%1.1f%%',
                    textprops={'color': 'black', 'fontsize': 10},
                    radius=0.75,
                    startangle=90
                )
                ax_pie.axis("equal")
                st.pyplot(fig_pie)

            # SCATTER CHART
            with g4:
                st.markdown("""
                <div style='background:#ffffff20; padding:15px; border-radius:15px;'>
                    <h3 style='text-align:center; color:black;'>
                     Consistency Scatter Plot
                    </h3>
                </div>
                """, unsafe_allow_html=True)

                fig, ax = plt.subplots(figsize=(6, 4))
                fig.patch.set_alpha(0)
                ax.set_facecolor("none")

                for subject in df_logs['subject'].unique():
                    sub_df = df_logs[df_logs['subject'] == subject]

                    ax.scatter(
                        sub_df['hours'],
                        sub_df['quiz_score'],
                        s=80,
                        alpha=0.85,
                        label=subject
                    )

                ax.set_xlabel("Study Hours")
                ax.set_ylabel("Quiz Score")
                ax.legend(title="Subject", fontsize=8)
                ax.grid(True, linestyle="--", alpha=0.3)

                st.pyplot(fig)

            st.success("✅ Dashboard Loaded Successfully!")

    if st.button("Logout"):
        st.session_state["page"] = "home"
        st.rerun()

Writing app.py


In [ ]:
!streamlit run app.py &>/dev/null &

In [ ]:
# FULL NGROK RESET
!rm -rf /root/.ngrok2
!rm -rf /root/.config/ngrok
!rm -rf ~/.ngrok2
!rm -rf ~/.config/ngrok

from pyngrok import ngrok
ngrok.kill()


In [ ]:
!ngrok config add-authtoken 34x7BDwOLuMqF2m9bJHquhAnRWo_59ABPzGXeHNoRT2SR4FiN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install streamlit pyngrok


In [ ]:
!streamlit run app.py --server.port 8501 &>/content/log.txt &


In [ ]:
from pyngrok import ngrok
ngrok.kill()

public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://margert-ablest-undespondingly.ngrok-free.dev" -> "http://localhost:8501">